In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow_model_optimization.quantization.keras import vitis_quantize
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.constraints import max_norm

from bmis_bci_utils import *

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score 
from sklearn import metrics
from sklearn.metrics import classification_report 

In [ ]:
input_size = (8, 50, 1)
no_classes = 7


def eeg_net(input_size, no_classes):
    
    inputs = tf.keras.Input(shape=input_size)
    x = tf.keras.layers.Conv2D(8, 16, padding='same', input_shape=input_size)(inputs)
    x = tf.keras.layers.Conv2D(32, 8, activation="relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((1,4))(x)
    x = tf.keras.layers.Conv2D(16, 16, activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x  =tf.keras.layers.MaxPooling2D((1,8))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(150, activation='relu')(x)
    x = tf.keras.layers.Dense(25, activation='relu')(x)
    outputs = tf.keras.layers.Dense(7, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="BMIS-BCI-NET")
    
    return model



def run_eeg_net_experiment(start_subject=1, stop_subject=33, split_ratio=0.2, no_gesture=7, fs=250, notch_freq=60.0, 
                    quality_factor=30.0, fc=10.0, fh=50.0,
                  order=5, window_time=200, overlap=60, no_channel=8, opt='adam', 
                   ls='sparse_categorical_crossentropy', mtr='accuracy', n_batches=16, n_epochs=30):
    
    result = pd.DataFrame({
        'Subject': [0],
        'Validation_result':[0.0],
        'qat_Validation_result':[0.0],
        'Test_result_32_bit':[0.0],
        'Test_result_8_bit_ptq':[0.0],
        'Test_result_8_bit_qat':[0.0],
        'Kappa_Score':[0.0],
        
        'precision_full':[0.0],
        'recall_full':[0.0],
        'f1_score_full':[0.0],

        'precision_ptq':[0.0],
        'recall_ptq':[0.0],
        'f1_score_ptq':[0.0],
        
        'precision_qat':[0.0],
        'recall_qat':[0.0],
        'f1_score_qat':[0.0]
        })
    
    '''
    data, label = get_data_subject_specific(start_subject)
    label = label.reshape(-1)
    no_classes = len(np.unique(label))

    #print('The total data shape is {} and label is {}'.format(data.shape, label.shape))

    X, y = window_with_overlap(data, label, sampling_frequency=fs, window_time=window_size, 
                               overlap=overlap, no_channel=no_channel)
    #print('The total Input data shape after windowing is {} and label is {}'.format(X.shape, y.shape))

    X_train, y_train, X_test, y_test = spilt_data(X, y, ratio=split_ratio)
    #print('Training Set is{} Test Set {}'.format(X_train.shape, X_test.shape))
    
    X_train, y_train, X_test, y_test = spilt_data(X, y, ratio=split_ratio)
    print('Training Set is{} Test Set {}'.format(X_train.shape, X_test.shape))
        
    X_train = np.expand_dims(X_train, axis=3)
    X_test = np.expand_dims(X_test, axis=3)
    input_size = X_train.shape[1:]
    
    '''


    for subject in range(start_subject, (stop_subject+1)):
        
        fold_no = 1
        num_folds = 3
        accuracy_per_fold = []
        loss_per_fold = []
        
        accuracy_per_fold=[]
        loss_per_fold=[]
        
        qat_accuracy_per_fold = []
        qat_loss_per_fold = []
        kfold = KFold(n_splits=num_folds, shuffle=False)
        
        
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        print(f'Training and Evaluation for subject {subject} with window size {200}')
        print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')


        data, label = get_data_subject_specific(subject)
        label = label.reshape(-1)
        no_classes = len(np.unique(label))

        #print('The total data shape is {} and label is {}'.format(data.shape, label.shape))

        X, y = window_with_overlap(data, label, sampling_frequency=fs, window_time=window_time, 
                                   overlap=overlap, no_channel=no_channel)
        #print('The total Input data shape after windowing is {} and label is {}'.format(X.shape, y.shape))

        X_train, y_train, X_test, y_test = spilt_data(X, y, ratio=split_ratio)
        #print('Training Set is{} Test Set {}'.format(X_train.shape, X_test.shape))

        X_train, y_train, X_test, y_test = spilt_data(X, y, ratio=split_ratio)
        print('Training Set is{} Test Set {}'.format(X_train.shape, X_test.shape))

        X_train = np.expand_dims(X_train, axis=3)
        X_test = np.expand_dims(X_test, axis=3)
        input_size = X_train.shape[1:]
        
        calibration_dataset = X_train[0:100] # Calibration data needed to quantize the model
        print(f'Input shape to the EEG-Net Model is: {input_size}')
        
        for train, test in kfold.split(X_train, y_train):
    
            early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)
            checkpoint_path = os.path.join('../checkpoint/full/200', str(subject))

            if not os.path.exists(checkpoint_path):
                os.makedirs(checkpoint_path)


            checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                        filepath = checkpoint_path, save_best_only=True,
                                        monitor='accuracy', vebrose=1)
    

            model_200 = eeg_net(input_size, no_classes)
            model_200.compile(optimizer=opt, loss=ls, metrics=mtr)

            print('---------------------------------------------------')
            print(f'Training for fold {fold_no} -------')

            history = model_200.fit(X_train[train], y_train[train], callbacks=[early_stop, checkpoint_callback],
                                batch_size=n_batches, epochs= n_epochs, verbose=1)

            scores = model_200.evaluate(X_train[test], y_train[test], verbose=0)
            print(f'Score for fold  {fold_no}: {model_200.metrics_names[0]} of {scores[0]}; {model_200.metrics_names[1]} of {scores[1]*100}%')
            accuracy_per_fold.append(scores[1] *100)
            loss_per_fold.append(scores[0])
          
            
            
            print('######################################################')
            print('Quantization Aware Training')
            
            qat_checkpoint_path = os.path.join('../checkpoint/qat/200', str(subject))

            if not os.path.exists(qat_checkpoint_path):
                os.makedirs(qat_checkpoint_path)
     

            qat_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                        filepath = qat_checkpoint_path, save_best_only=True, 
                                        monitor='accuracy', vebrose=1)
            
            qat_model = eeg_net(input_size, no_classes)
            quantizer = vitis_quantize.VitisQuantizer(qat_model, quantize_strategy='8bit_tqt')
            qat_model_set = quantizer.get_qat_model(init_quant=True, calib_dataset=calibration_dataset)

            qat_model.compile(optimizer= opt, loss=ls, metrics=mtr)

            print('---------------------------------------------------')
            print(f'Quantization Aware Training for fold {fold_no} -------')

            qat_history = qat_model.fit(X_train[train], y_train[train],
                                        callbacks=[early_stop, qat_checkpoint_callback], 
                                        batch_size=n_batches, epochs= n_epochs, verbose=1)

            qat_scores = qat_model.evaluate(X_train[test], y_train[test], verbose=0)
            print(f'QAT Score for fold  {fold_no}: {qat_model.metrics_names[0]} of {qat_scores[0]}; {qat_model.metrics_names[1]} of {qat_scores[1]*100}%')
            qat_accuracy_per_fold.append(qat_scores[1] *100)
            qat_loss_per_fold.append(scores[0])

            fold_no = fold_no + 1
        
        print("Average Full Bit Validation Score per fold ")

        for i in range(0, len(accuracy_per_fold)):
            print('-----------------------------------------------')
            print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {accuracy_per_fold[i]}%')
        print('-----------------------------------------------')
        print('Average Metrics for all folds: ')
        print(f'> Accuracy: {np.mean(accuracy_per_fold)} (+- {np.std(accuracy_per_fold)})')
        print(f'> Loss: {np.mean(loss_per_fold)}')
        print('-----------------------------------------------')
        print('############################################################')
        print(f'Training Ended for subject {subject}')
        print('############################################################')
        
        validation_result = np.mean(accuracy_per_fold)
        qat_validation_result = np.mean(qat_accuracy_per_fold)
        
        ######### Load the best model from checkpoint. ################
        model = tf.keras.models.load_model(checkpoint_path)
        
        ############# Evaluating 32-bit Model ##########################
        print('############################################################')
        print(f'Evaluating Unquantized Model')
        print('############################################################')
        uqt_test_loss, uqt_test_accuracy = model.evaluate(X_test, y_test)
        full_test_accuracy = uqt_test_accuracy * 100
        print(f'Accuracy of the Unquantized_model {full_test_accuracy}%')
        
        
        y_predict = model.predict(X_test)
        y_predict = np.argmax(y_predict, axis=-1)
   
        precision = precision_score(y_test, y_predict, average='weighted')
        recall = recall_score(y_test, y_predict, average='weighted')
        f1_score_full = 2 * (precision * recall) / (precision + recall)
        kappa_score = cohen_kappa_score(y_test, y_predict)


        
        print(f'Precision of the Unquantized model {precision}')
        print(f'Recall of the Unquantized model {recall}')
        print(f'F1_score of the Unquantized model {f1_score_full}')
        print(f'Kappa_score of the Unquantized model {kappa_score}')
    
        
        ############ Save the full model as .h5. This will be used for quantization #####
        saved_model = '../full_models/200/EEG-NET-200-' + str(subject) + '.h5'
        model.save(saved_model)
        
        
        
        ############### Post-Training Quantization ########################
        saved_float32_model = tf.keras.models.load_model(saved_model)
        ptq_quantizer = vitis_quantize.VitisQuantizer(saved_float32_model)
        ptq_model = ptq_quantizer.quantize_model(calib_dataset=calibration_dataset)
        
        
        ############# Evalauting PTQ Model #######################################
        print('############################################################')
        print(f'Evaluating PTQ Model')
        print('############################################################')

        ptq_model.compile(loss=ls, metrics=mtr)
        ptq_loss, ptq_test_accuracy = ptq_model.evaluate(X_test, y_test)
        ptq_test_accuracy = ptq_test_accuracy * 100
        print(f'Accuracy of the PTQ model {ptq_test_accuracy}%')
        
        
            
        ptq_y_predict = ptq_model.predict(X_test)
        ptq_y_predict = np.argmax(ptq_y_predict, axis=-1)
        ptq_precision = precision_score(y_test, ptq_y_predict, average='weighted')
        ptq_recall = recall_score(y_test, ptq_y_predict, average='weighted')
        ptq_f1_score = 2 * (ptq_precision * ptq_recall) / (ptq_precision + ptq_recall)
        
        
        print(f'Precision of the PTQ model {ptq_precision}')
        print(f'Recall of the PTQ model {ptq_recall}')
        print(f'F1_score of the PTQ model {ptq_f1_score}')

        # Saving PTQ model; Can be complied for depolyment #######  
        ptq_quantized_model = '../ptq_models/200/ptq-EEG-NET-' + str(subject) + '.h5'
        ptq_model.save(ptq_quantized_model)
        
        
        ############# Evalauting QAT Model ###################################
        
        ## Load Best QAT Model
        qat_model = tf.keras.models.load_model(qat_checkpoint_path)
        
        ## ######### Evaluate QAT Model###############################################
        print('############################################################')
        print(f'Evaluating QAT Model')
        print('############################################################')
        
        qat_test_loss, qat_test_accuracy = qat_model.evaluate(X_test, y_test)
        qat_test_accuracy = qat_test_accuracy * 100
        print(f'Accuracy of the QAT model {qat_test_accuracy}%')
        
        
        qat_y_predict = qat_model.predict(X_test)
        qat_y_predict = np.argmax(qat_y_predict, axis=-1)
        qat_precision = precision_score(y_test, qat_y_predict, average='weighted')
        qat_recall = recall_score(y_test, qat_y_predict, average='weighted')
        qat_f1_score = 2 * (qat_precision * qat_recall) / (qat_precision + qat_recall)
        
        
        print(f'Precision of the QAT model {qat_precision}')
        print(f'Recall of the QAT model {qat_recall}')
        print(f'F1_score of the QAT model {qat_f1_score}')
        
        # Saving QAT model; Can be complied for depolyment #######  
        qat_quantized_model = '../qat_models/200/qat-EEG-NET-' + str(subject) + '.h5'
        qat_model.save(qat_quantized_model)
        
        
        result.at[subject-1, 'Subject'] = subject
        result.at[subject-1, 'Validation_result'] = validation_result
        result.at[subject-1, 'qat_Validation_result'] = qat_validation_result
        result.at[subject-1, 'Test_result_32_bit'] = full_test_accuracy
        result.at[subject-1, 'Test_result_8_bit_ptq'] = ptq_test_accuracy
        result.at[subject-1, 'Test_result_8_bit_qat'] = qat_test_accuracy
        result.at[subject-1, 'Kappa_Score'] = kappa_score      
        
        result.at[subject-1, 'precision_full'] = precision 
        result.at[subject-1, 'recall_full'] = recall
        result.at[subject-1, 'f1_score_full'] = f1_score_full
   
        result.at[subject-1, 'precision_ptq'] = ptq_precision 
        result.at[subject-1, 'recall_ptq'] = ptq_recall
        result.at[subject-1, 'f1_score_ptq'] = ptq_f1_score
        
        result.at[subject-1, 'precision_qat'] = qat_precision 
        result.at[subject-1, 'recall_qat'] = qat_recall
        result.at[subject-1, 'f1_score_qat'] = qat_f1_score

        save_path = str(start_subject)+'_to_'+str(stop_subject)+'_EEG_Net.csv'
        save_path = os.path.join('../results/200', save_path)
        result.to_csv(save_path, index=False)

In [ ]:
run_eeg_net_experiment(start_subject=12, stop_subject=33, split_ratio=0.2, no_gesture=7, fs=250, notch_freq=60.0, 
                    quality_factor=30.0, fc=10.0, fh=50.0,
                    order=5, window_time=200, overlap=60, no_channel=8, opt='adam', 
                    ls='sparse_categorical_crossentropy', mtr='accuracy', n_batches=16, n_epochs=50)

In [ ]:
!vai_c_tensorflow2 \
    --model ../ptq_models/200/ptq-EEG-NET-33.h5 \
    --arch ./arch_ultra96.json \
    --output_dir ../inference_models \
    --net_name subject_33_model 